from [Likelihood encoding of categorical features
](https://www.kaggle.com/tnarik/likelihood-encoding-of-categorical-features)

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.model_selection import StratifiedKFold, KFold

In [3]:
df_train = pd.read_pickle('../input/feats/df_trn_feat0.pkl')
df_test  = pd.read_pickle('../input/feats/df_test_feat0.pkl')

In [4]:
#### params ###

# np.random.seed(13)
# target = 'deal_probability'
# feature = 'param_1'
# data = df_train.iloc[:10000,:].copy()

#####
def impact_coding(data, feature, target='deal_probability', seed=0):
    ## most of the code from: https://www.kaggle.com/tnarik/likelihood-encoding-of-categorical-features
    
    n_folds = 5
    n_inner_folds = 4
    print('encode:{}\ttarget:{}'.format(feature,target))
    print('n_folds:{},n_inner_folds:{}'.format(n_folds,n_inner_folds))
    impact_coded = pd.Series()

    oof_default_mean = data[target].mean() # Gobal mean to use by default

    kf = KFold(n_splits=n_folds, shuffle=True,random_state=seed)

    oof_mean_cv = pd.DataFrame()
    split = 0

    for infold_no, (infold, oof) in enumerate(kf.split(data[feature])):
        print('encode infold no:',infold_no)
        impact_coded_cv = pd.Series()
        kf_inner = KFold(n_splits=n_inner_folds, shuffle=True,random_state=seed)
        inner_split = 0
        infold_inner_mean_cv = pd.DataFrame() ## inner_oof_mean_cv <-> infold_inner_mean_cv
        oof_default_inner_mean = data.iloc[infold][target].mean()

        for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
            # The mean to apply to the inner oof split   (a 1/n_folds % based on the rest)
            infold_inner_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()

            ############# 不清楚用途???? ##################
#             impact_coded_cv_tmp = data.iloc[infold].copy()
#             impact_coded_cv_tmp = impact_coded_cv_tmp[feature].map(oof_mean).fillna(oof_default_inner_mean)
#             impact_coded_cv = impact_coded_cv.append(impact_coded_cv_tmp)
            #############################################

            # Also populate mapping (this has all group -> mean for all inner CV folds)
            infold_inner_mean_cv = infold_inner_mean_cv.join(pd.DataFrame(infold_inner_mean), rsuffix=inner_split, how='outer') ## inner_oof_mean_cv <-> infold_inner_mean_cv 
            infold_inner_mean_cv.fillna(value=oof_default_inner_mean, inplace=True) ## inner_oof_mean_cv <-> infold_inner_mean_cv  
            inner_split += 1

        # Also populate mapping
        oof_mean_cv = oof_mean_cv.join(pd.DataFrame(infold_inner_mean_cv), rsuffix=split, how='outer')
        oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
        split += 1

        #### map to oof ##############################
        impact_coded_tmp = data.iloc[oof].copy()
        infold_inner_mean_cv_ = infold_inner_mean_cv.sum(axis=1) ## oof_inner_mean
        impact_coded_tmp = impact_coded_tmp[feature].map(infold_inner_mean_cv_).fillna(oof_default_mean)        
        impact_coded = impact_coded.append(impact_coded_tmp)
        #############################################

    return impact_coded.sort_index(), oof_mean_cv.mean(axis=1), oof_default_mean

____
for test purpose


In [45]:
#### params ###
np.random.seed(13)
target = 'deal_probability'
feature = 'param_1'
data = df_train.iloc[:10000,:].copy()


n_folds = 5
n_inner_folds = 4

impact_coded = pd.Series()

oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)

kf = KFold(n_splits=n_folds, shuffle=True)

oof_mean_cv = pd.DataFrame()
split = 0

#####

for infold, oof in kf.split(data[feature]):
    impact_coded_cv = pd.Series()
    kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
    inner_split = 0
    infold_inner_mean_cv = pd.DataFrame() ## inner_oof_mean_cv <-> infold_inner_mean_cv
    oof_default_inner_mean = data.iloc[infold][target].mean()
    
    for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
        # The mean to apply to the inner oof split   (a 1/n_folds % based on the rest)
        infold_inner_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()
        
        ############# 不清楚用途???? ##################
        impact_coded_cv_tmp = data.iloc[infold].copy()
        impact_coded_cv_tmp = impact_coded_cv_tmp[feature].map(oof_mean).fillna(oof_default_inner_mean)
        impact_coded_cv = impact_coded_cv.append(impact_coded_cv_tmp)
        #############################################

        # Also populate mapping (this has all group -> mean for all inner CV folds)
        infold_inner_mean_cv = infold_inner_mean_cv.join(pd.DataFrame(infold_inner_mean), rsuffix=inner_split, how='outer') ## inner_oof_mean_cv <-> infold_inner_mean_cv 
        infold_inner_mean_cv.fillna(value=oof_default_inner_mean, inplace=True) ## inner_oof_mean_cv <-> infold_inner_mean_cv  
        inner_split += 1

    # Also populate mapping
    oof_mean_cv = oof_mean_cv.join(pd.DataFrame(infold_inner_mean_cv), rsuffix=split, how='outer')
    oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
    split += 1
    
    ##### 不清楚用途 ###########################################
    impact_coded_tmp = data.iloc[oof].copy()
    infold_inner_mean_cv_ = infold_inner_mean_cv.sum(axis=1)
    impact_coded_tmp = impact_coded_tmp[feature].map(infold_inner_mean_cv_).fillna(oof_default_mean)
    impact_coded = impact_coded.append(impact_coded_tmp)
    #############################################

